In [1]:
# Importar librerías
import pandas as pd 
import numpy as np
import calendar
import locale
import json
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



# Establecer el idioma en español
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')


'es_ES.UTF-8'

In [2]:
# Cargar el csv a utilizar y ponerlo en un df
df = pd.read_csv("Dataset/movies_final.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45372 entries, 0 to 45371
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45370 non-null  float64
 1   genres                45370 non-null  object 
 2   id                    45370 non-null  object 
 3   original_language     45359 non-null  object 
 4   overview              44429 non-null  object 
 5   popularity            45368 non-null  object 
 6   production_countries  45370 non-null  object 
 7   release_date          45367 non-null  object 
 8   revenue               45370 non-null  float64
 9   runtime               45121 non-null  float64
 10  spoken_languages      45367 non-null  object 
 11  status                45287 non-null  object 
 12  tagline               20395 non-null  object 
 13  title                 45367 non-null  object 
 14  vote_average          45367 non-null  float64
 15  release_year       

C:\Users\cora1\AppData\Local\Temp\ipykernel_31140\2221441072.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset/movies_final.csv")


********** FUNCIONES **********

# 1er función:
def peliculas_mes(mes): '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
return {'mes':mes, 'cantidad':respuesta}

In [3]:
def peliculas_mes(mes):
        # Convertir la columna "release_date" de object a datetime
        df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce') # Se reemplaza las fechas dejando de lado los errores
        # Crear la columna release_month donde se extraerá el mes de la fecha de estreno.
        df['release_month'] = df['release_date'].apply(lambda x: x.month if x is not pd.NaT else None)
        df['release_month'] = df['release_month'].fillna(0).astype(int)
        meses = dict(enumerate(calendar.month_name))
        df['release_month'] = df['release_month'].map(meses)
        #meses = { i+1: calendar.month_name[i+1] for i in range(12) }
        #meses
        cantidad = df[df["release_month"] == mes]["release_month"].count()
        return f'Mes: {mes}, Cantidad: {cantidad}'

peliculas_mes('octubre')

'Mes: octubre, Cantidad: 4614'

# 2da Función:
def peliculas_dia(dia): '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia historicamente''' return {'dia':dia, 'cantidad':respuesta}

In [4]:
def peliculas_dia(dia):
    # Convertir la columna "release_date" de object a datetime
    df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce') # Se reemplaza las fechas dejando de lado los errores
        
    # Crear la columna release_day donde se extraerá el día de la fecha de estreno
    # df['release_day'] = df['release_date'].apply(lambda x: x.day if x is not pd.NaT else None)
    # df['release_day'] = df['release_day'].fillna(0).astype(int)
    #Para sacar la columna días de la semana:

    df["release_day"] = df['release_date'].dt.strftime('%A').apply(lambda x: x if type(x) != float else x)
    df["release_day"]
    respuesta = df[df["release_day"] == dia]["release_day"].count()
    return f'Día: {dia}, Cantidad: {respuesta}' 

peliculas_dia('martes')


'Día: martes, Cantidad: 4640'

# 3ra Función:
def franquicia(franquicia): '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio''' return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}


In [5]:
def franquicia(franquicia):
    # Filtramos las filas que pertenecen a la franquicia deseada y contamos cuántas filas hay en el resultado filtrado.
    cantidad = df[df['name'] == franquicia].shape[0]
    
    df_franquicia = df[df['name'] == franquicia] # Filtra la franquicia solicitada

    # Calculamos la ganancia total
    gananciaTotal = (df_franquicia['revenue'] - df_franquicia['budget']).sum()
    gananciaPromedio = (df_franquicia['revenue'] - df_franquicia['budget'] / df_franquicia['budget']).mean()

    return f'Franquicia: {franquicia}, Cantidad: {cantidad}, Ganancia Total: {gananciaTotal}, Ganancia Promedio: {gananciaPromedio}'

franquicia("Toy Story Collection")

'Franquicia: Toy Story Collection, Cantidad: 3, Ganancia Total: 343554033.0, Ganancia Promedio: 373554032.0'

# 4ta Función:
def peliculas_pais(pais): '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo''' return {'pais':pais, 'cantidad':respuesta}

In [6]:
def peliculas_pais(pais):
    
    mask = df['production_countries'].str.join(',').str.contains(pais, na=False)
   
    can = mask.count()
    # cantidad = df[mask]['title'].count()

    return f'pais: {pais}, cantidad:{can}'

peliculas_pais('United States of America')

'pais: United States of America, cantidad:45372'

# 5ta Función:
def productoras(productora): '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron''' return {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}

In [7]:
def productoras(productora):
    prod = df[['name_production', 'budget', 'revenue']].dropna()
    prod ['name_production'] = prod['name_production'].map(str.lower)
    cantidad = prod.shape[0]
    gtotal= (prod['revenue'] - prod['budget']).sum()
    return {'productora':productora, 'ganancia_total': gtotal, 'cantidad': cantidad } 

productoras('Warner Bros.')

{'productora': 'Warner Bros.',
 'ganancia_total': 317932254772.0,
 'cantidad': 33573}

# Función 6
def retorno(pelicula): '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo''' return {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}

In [8]:
def retorno(pelicula):
    pelicula_data = df[df['title'] == pelicula]
    if len(pelicula_data) == 0:
        return f"No se encontró la película {pelicula}"
    inversion = pelicula_data['budget'].iloc[0]
    ganancia = pelicula_data['revenue'].iloc[0] - inversion
    retorno = ganancia / inversion if inversion > 0 else 0
    anio = int(pelicula_data['release_year'])
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio}

retorno('Toy Story')

{'pelicula': 'Toy Story',
 'inversion': 30000000.0,
 'ganancia': 343554033.0,
 'retorno': 11.4518011,
 'anio': 1995}

# Función 7 ML
def recomendacion('titulo'): '''Ingresas un nombre de película y te recomienda las similares en una lista de 5 valores''' return {'lista recomendada': respuesta}

In [9]:
def recomendacion(titulo): # Toma como parámetro el título de una película y retorna una lista de 5 películas recomendadas de acuerdo al título 
    df = pd.read_csv('Dataset/movies_ML.csv')
    
    tfidf = TfidfVectorizer(stop_words = 'english') #  Crea una instancia del objeto TfidfVectorizer que se utiliza para calcular la matriz
    df['overview'] = df['overview'].fillna('') # Rellena los valores faltantes (nulos) en la columna 'overview' del DataFrame 'df' con una cadena vacía ('')

    tfidf_matriz = tfidf.fit_transform(df['overview'])
    coseno_sim = linear_kernel(tfidf_matriz, tfidf_matriz) # Se utiliza para calcular la similitud de coseno
                                                           # La similitud de coseno es una medida comúnmente utilizada en procesamiento de texto para determinar la similitud entre dos documentos basándose en sus representaciones vectoriales
    indices = pd.Series(df.index, index=df['title']).drop_duplicates()
    idx = indices[titulo] # Obtiene el índice correspondiente a un título de película específico en el objeto Series de índices creado anteriormente

    # Obtener el puntaje de similitud de esa pelicula con todas las películas
    simil = list(enumerate(coseno_sim[idx]))

    # Ordena las películas de acuerdo a su puntaje de similitud
    simil = sorted(simil, key=lambda x: x[1], reverse =True)

    # Obtiene el puntaje de similitud de 10 películas
    simil = simil[1:11]
    
    # Obtiene los índices 
    movie_index = [i[0] for i in simil]

    # Retorna 5 películas
    lista = df['title'].iloc[movie_index].to_list()[:5] 
      
    return {'lista recomendada': lista}

recomendacion('Toy Story')

{'lista recomendada': ['Toy Story 2',
  'The Champ',
  'Rebel Without a Cause',
  'Man on the Moon',
  'Malice']}